In [1]:
from py_ecc.bls.g2_primitives import pubkey_to_G1, signature_to_G2

In [2]:
from py_ecc.bls12_381 import G1, G2, multiply, pairing, neg, curve_order
from py_ecc.optimized_bls12_381 import add, Z1
import os
import time
import hashlib

In [3]:
import py_ecc
from py_ecc.fields import (
    bls12_381_FQ as FQ,
    bls12_381_FQ2 as FQ2,
    bls12_381_FQ12 as FQ12,
    bls12_381_FQP as FQP,
)

In [4]:
# Simulate a private-public key pair
private_key = int.from_bytes(os.urandom(32), 'big') % curve_order
public_key = multiply(G1, private_key)  # [private_key]G2

In [144]:
def blsSign(message, private_key):
    # Message hash (simplified, in practice use a secure hash-to-curve function)
    message_hash = multiply(G2, message)

    # Sign: signature = [private_key]H(m)
    signature = multiply(message_hash, private_key)

    return signature

In [145]:
def blsVerify(public_key, message, signature):
    message_hash = multiply(G2, message)
    return pairing(message_hash, public_key) == pairing(neg(signature),neg(G1))

In [11]:
GT = pairing(G2,G1)

In [146]:
# Hash-to-curve function (simplified, maps to G1)
def hash_to_G1(message):
    # Hash the message to a scalar and map to G1
    # In practice, use a secure hash-to-curve
    # This is a simplified version for prototyping
    h = hashlib.sha256(str(message).encode()).digest()
    scalar = int.from_bytes(h, 'big') % curve_order
    return multiply(G1, scalar)

In [203]:
signature = blsSign(20520820,private_key)

In [204]:
message = 20520820
print("pk", public_key)
print("sig",signature)
print("H(m)",hash_to_G1(20520820))
message_hash = multiply(G2, message)
blsVerify(public_key, message, signature)

pk (2288140669805754146034429088305634003924644169851294203779919219214088538782878606945747948470635299103273815362397, 1546641121591883163939834989364703750667943713267062029485373499338319203859772948340193697688762734743790625048749)
sig ((3896799623892883481180825861067839853763663138197116470781961435031082203423175200183138975925120539526534106318735, 3353502529166151086170773251300610240842510249314973639886096359992398292581470598355900145280337666143908979608954), (1798859856937516732007337673371806949025314255672882151926218279700656703103079417960534703190494642906198611842805, 1763012569307665510099512280787932578593046079065706186030147288098501701696680102784133220848882642856635263727086))
H(m) (2678103951128236434036528609166284486865305787838041906061161155718261300389561534985571943727676808509120531122627, 47959504814261139810716729533368699644622975911391729082320392825090231633727900096161277890225592306651577000391)


True

In [19]:
# Setup: Generate keys
def setup():
    # Private key: random scalar
    private_key = int.from_bytes(os.urandom(32), 'big') % curve_order
    # Public key: [private_key]G2
    public_key = multiply(G2, private_key)
    return private_key, public_key

In [9]:
# Encryption: Enc(m, t, pk, g1)
def encrypt(message, time, public_key, g1=G1):
    if not isinstance(message, int) or message < 0:
        raise ValueError("Message must be a non-negative small integer")
    # Hash time to G2 (in production use proper hash-to-curve function)
    h_t = multiply(G2,time)
    # Random scalar r
    r = int.from_bytes(os.urandom(32), 'big') % curve_order
    # c1 = g1^(m + r)
    c1 = multiply(g1, r % curve_order)
    # c2 = e(h_t, pk)^r
    g_rho = pairing(h_t, public_key)
    # Raise e_ht_pk to the power r (in GT, which is a field)
    # py_ecc doesn't provide direct GT exponentiation, so we simulate
    # In practice, use a library with GT operations or adjust scheme
    # For simplicity, store e_ht_pk and r (not secure, for demo only)
    c2 = (GT**message)*(g_rho**r)
    return c1, c2

In [252]:
## The secret and public key of the authority who signs the timestamps
sk, pk = setup()
print("Let's encrypt!")
noOfEncryptedMessages = 10
start = time.time()
for i in range(noOfEncryptedMessages):
    c1,c2=encrypt(i+5,20250820,pk)
end = time.time()
print("Encrypting one message takes: ",(end - start)/noOfEncryptedMessages)  

Let's encrypt!


TypeError: Expected an FQP object, but got object of type <class 'py_ecc.fields.bls12_381_FQ'>

In [196]:
def createLookUpTable(noOfEncryptedMessages):
    table = {}
    for i in range(noOfEncryptedMessages):
        GTm = GT**i
        table[hashlib.sha256(str(GTm.coeffs[0]).encode()).digest()]=i
    return table

In [197]:
table = createLookUpTable(2048)

In [198]:
hashlib.sha256(str(GT.coeffs[0]).encode()).digest()

b'\xc5\x92\xb5#57\xb0\xa5@L\xa1\xee4\xfb9\x03\x83&Ej\x972\xae\xdc\xe2\x03\xb2\xa9%%\xcb\x92'

In [232]:
# Decryption: Dec(c1, c2, t, sigma_t, g2)
def decrypt(c1, c2, time, signature, table, public_key):
    #print(blsVerify(public_key, time, signature))
    gTm = c2*pairing(neg(signature), c1)
    #print(gTm)
    keyOfM = hashlib.sha256(str(gTm.coeffs[0]).encode()).digest()
    m = table[keyOfM]
    return m

In [13]:
def homomorphicAddition(c11, c12, c21, c22):
    c31 = py_ecc.bls12_381.add(c11, c21)
    c32 = c12*c22 
    return (c31, c32)

In [222]:
(c1,c2)=encrypt(43,20520820,public_key)

In [223]:
(c3,c4)=encrypt(5,20520820,public_key)

In [224]:
c5,c6 = homomorphicAddition(c1,c2,c3,c4)

In [225]:
decrypt(c5,c6,20520820,signature,table,public_key)

True
(2489785276631328532382142120316311246421586111889668427669664432779815905670575890864251129229565078115263922597179, 1140382248321545103566733409060785179279785875532383357925979206339050297359063098560734115941687929474943168643441, 2212276760027287352708487584443730086020194872786412390215096450409209743811230500833359062492032453937809873415821, 1983090043190328134757268627700957737926235620877492902545811290820092442709714266416995940432193144160546576429165, 2025720272019788713742928880389510520246709349929780306200708705488480397978260897241433383898826053741861599263410, 3029361848257132684841214088469464377900090561313144047514894752649602007890322301777868911287565471872456061945535, 392010473858789823983567521442395367009907228419650636440882166359987668135456994190147727125473436144726676039927, 2065784563995303653592069024021957129204640725037314518170587968716444932781664550707250792920888131543107525641957, 30885264985639015509983441261664000752352476595544731879281

48

In [167]:
GT**43

(3075181513924604203747816211905240043391475707871695446958625655841442090780725342058643353154594722075857509823206, 1552044913488320305383075166329358337178514531188939793195137978864084373258324081604374890056817311297726434722219, 2001455281950980003234726740770617078177441893920639654463875656501298268243067863739996352385272135522296190123676, 1690247094084382516084964823749500435580313678219710543398121378073254324148644418138984038949827116695602294527568, 3679985777122218733312892402635664565995939705497953266129447819942841149753689434725062032969682725067154591791268, 3700277798497245412424581448036267041689856247537014098411638078317892191354475602216372076541041228898266061764766, 120851743802075313597062557165364294569143517026561492445696544816107290141973742779406724059757034085837233478066, 631010317016923343636817313740414360249950807431926429435145953839484758949653053241346270081179418954155680435884, 37297519319955298882727150006281766809519080317157519758247370217

In [253]:
private_key = int.from_bytes(os.urandom(32), 'big') % curve_order
public_key = multiply(G1, private_key)  # [private_key]G2
signature = blsSign(20520820,private_key)
benchmarkSize = 4
encryptionTimes = []
decryptionTimes = []
for j in range(17,20):
    tableSize = 2**j
    table = createLookUpTable(tableSize)
    print("The "+str(j)+"th lookup table is ready")
    encryptionTime = 0
    decryptionTime = 0
    print(j)
    for i in range(benchmarkSize):
        message = tableSize-1-i*(tableSize//benchmarkSize)
        start = time.time()
        (c1,c2)=encrypt(message,20520820,public_key)
        end = time.time()
        encryptionTime+=(end-start)
        start = time.time()
        decrypt(c1,c2,20520820,signature,table,public_key)
        end = time.time()
        decryptionTime+=(end-start)
    print(encryptionTime/benchmarkSize,decryptionTime/benchmarkSize)
    encryptionTimes.append(encryptionTime/benchmarkSize)
    decryptionTimes.append(decryptionTime/benchmarkSize)

The 17th lookup table is ready
17
2.087894022464752 1.9509211778640747


KeyboardInterrupt: 

In [ ]:
print(encryptionTimes)

In [244]:
print(decryptionTimes)

[1.92889004945755, 1.9304082989692688, 1.9462448954582214, 1.946574866771698, 1.9762815833091736, 1.9288567900657654, 1.9295653104782104, 1.9387152194976807, 1.9209685921669006, 1.94856196641922, 1.929402470588684, 1.9461219906806946, 1.9316332340240479, 1.9411841034889221]


In [15]:
## gotta have a measurement about the time it takes to homomorphically add HSWE ciphertexts
homomorphicAdditionTimes = []
private_key = int.from_bytes(os.urandom(32), 'big') % curve_order
public_key = multiply(G1, private_key)  # [private_key]G2
for i in range(1,14):
    (c1,c2)=encrypt(43,20520820,public_key)
    (c3,c4)=encrypt(43654,20520820,public_key)
    start = time.time()
    for j in range(2**i):
        c3,c4 = homomorphicAddition(c1,c2,c3,c4)
    end = time.time()
    print(2**i, end - start)
    homomorphicAdditionTimes.append(end-start)

2 0.000469207763671875
4 0.0009741783142089844
8 0.0019681453704833984
16 0.003941059112548828
32 0.007869243621826172
64 0.01544189453125
128 0.031036853790283203
256 0.061914920806884766
512 0.12740302085876465
1024 0.24971914291381836
2048 0.4975550174713135
4096 0.9962918758392334
8192 2.006026029586792


In [16]:
print(homomorphicAdditionTimes)

[0.000469207763671875, 0.0009741783142089844, 0.0019681453704833984, 0.003941059112548828, 0.007869243621826172, 0.01544189453125, 0.031036853790283203, 0.061914920806884766, 0.12740302085876465, 0.24971914291381836, 0.4975550174713135, 0.9962918758392334, 2.006026029586792]


In [ ]:
## RSA-Paillier scheme implementation asap

In [4]:
def random_prime(n):
    prime_candidate = 2
    while True:
            prime_candidate = getLowLevelPrime(n)
            if not isMillerRabinPassed(prime_candidate):
                continue
            else:
                break
    return prime_candidate

In [5]:
# Large Prime Generation for RSA
import random
 
# Pre generated primes
first_primes_list = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
                     31, 37, 41, 43, 47, 53, 59, 61, 67,
                     71, 73, 79, 83, 89, 97, 101, 103,
                     107, 109, 113, 127, 131, 137, 139,
                     149, 151, 157, 163, 167, 173, 179,
                     181, 191, 193, 197, 199, 211, 223,
                     227, 229, 233, 239, 241, 251, 257,
                     263, 269, 271, 277, 281, 283, 293,
                     307, 311, 313, 317, 331, 337, 347, 349]
 
 
def nBitRandom(n):
    return random.randrange(2**(n-1)+1, 2**n - 1)
 
 
def getLowLevelPrime(n):
    '''Generate a prime candidate divisible 
    by first primes'''
    while True:
        # Obtain a random number
        pc = nBitRandom(n)
 
        # Test divisibility by pre-generated
        # primes
        for divisor in first_primes_list:
            if pc % divisor == 0 and divisor**2 <= pc:
                break
        else:
            return pc
 
 
def isMillerRabinPassed(mrc):
    '''Run 30 iterations of Rabin Miller Primality test'''
    maxDivisionsByTwo = 0
    ec = mrc-1
    while ec % 2 == 0:
        ec >>= 1
        maxDivisionsByTwo += 1
    assert(2**maxDivisionsByTwo * ec == mrc-1)
 
    def trialComposite(round_tester):
        if pow(round_tester, ec, mrc) == 1:
            return False
        for i in range(maxDivisionsByTwo):
            if pow(round_tester, 2**i * ec, mrc) == mrc-1:
                return False
        return True
 
    # Set number of trials here
    numberOfRabinTrials = 30
    for i in range(numberOfRabinTrials):
        round_tester = random.randrange(2, mrc)
        if trialComposite(round_tester):
            return False
    return True

In [20]:
## Key Generation
p = random_prime(1024)
q = random_prime(1024)
e = 17
N = p*q
N2 = N*N
phiN = (p-1)*(q-1)
phiN2 = phiN*phiN
d = pow(e,-1,phiN2)
print("q",q)
print("p",p)
print("N",N)
print("d",d)
print("Key generation completed")

q 116611481445197203772824271007361947795379264339169615479567920769088265829104751086890395555777458354599893188514422425109660741664019996285011446869549494200608259856787170301106359688973678623479886301082018485375797685211552669388608189517203737820216210249602744776160646033893805538438910350615604858313
p 176667939693566499731178786144670647438661375705672633631256344669723543117232537166740867901690678434901096807008907916379926421775153376042104694433719474277121171647204074481939404995451198915583416340851957623521175843809169484611371838804096931583207508094997116222424094437080716337927920328737786140241
N 20601510171537548453520614606986485633504875365129215814887337475640063195359876543235474548068026823284021461452131226215223922462831390046980675561690033483616489521828564197199826243512834215409389458381217693897340766150140732553985166134148302945964153691492521985650950841826704772509324280520840384132850421427769156021681735164113729525472576971345228148934098902965

In [14]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [32]:
## Encryption algorithm
def encRSA(m,rho,N,N2,e):
    ## modular inverse: y = pow(x, -1, p)
    ## hashedRho = hashlib.sha256(str(rho).encode()).digest()
    hashedRho = rho
    r = int.from_bytes(os.urandom(32), 'big')
    U = (pow(1+N,m, N2)*pow(hashedRho,N*r*rho,N2))%N2
    V = pow(hashedRho, N*e*r*rho, N2)
    return (U,V)

In [75]:
def homomorphicAddRSA(U1,V1,U2,V2,N2):
    U3 = (U1*U2)%N2
    V3 = (V1*V2)%N2
    return (U3,V3)

In [21]:
def signRSA(V,d,N2):
    return pow(V,d,N2)

In [68]:
## Decryption algorithm
## Inputs: ciphertext U, V, rho, sigma (the RSA signature on the ciphertext)
def decRSA(U,V,rho,sigma,N,N2):
    W = (U*pow(sigma,-1,N2))%N2
    m = (W-1)//N
    return int(m)

In [34]:
(U,V)=encRSA(23,20250514,N,N2,e)

In [39]:
sigma = signRSA(V,d,N2)

In [44]:
W = decRSA(U,V,20250514,sigma)

In [72]:
## Bencharmking HSWE-RSA Encryption and Decryption times
benchmarkSize = 100
m =22654653454549669696964832929654654646464653464646466356432
encryptionTimes = 0
decryptionTimes = 0
for i in range(benchmarkSize):
    start = time.time()
    (U,V)=encRSA(m+i,20250514,N,N2,e)
    end = time.time()
    encryptionTimes+=(end-start)
    sigma = signRSA(V,d, N2)
    start = time.time()
    m1=decRSA(U,V,20250514,sigma, N, N2)
    end = time.time()
    print(i,m1==(m+i),m1)
    decryptionTimes+=(end-start)
print("Average encryption time:", encryptionTimes/benchmarkSize)
print("Average decryption time:", decryptionTimes/benchmarkSize)

0 True 22654653454549669696964832929654654646464653464646466356432
1 True 22654653454549669696964832929654654646464653464646466356433
2 True 22654653454549669696964832929654654646464653464646466356434
3 True 22654653454549669696964832929654654646464653464646466356435
4 True 22654653454549669696964832929654654646464653464646466356436
5 True 22654653454549669696964832929654654646464653464646466356437
6 True 22654653454549669696964832929654654646464653464646466356438
7 True 22654653454549669696964832929654654646464653464646466356439
8 True 22654653454549669696964832929654654646464653464646466356440
9 True 22654653454549669696964832929654654646464653464646466356441
10 True 22654653454549669696964832929654654646464653464646466356442
11 True 22654653454549669696964832929654654646464653464646466356443
12 True 22654653454549669696964832929654654646464653464646466356444
13 True 22654653454549669696964832929654654646464653464646466356445
14 True 22654653454549669696964832929654654646464653464646

In [77]:
## Benchmarking HSWE-RSA homomorphic addition for multiple ciphertexts
homomorphicAdditionTimes = []
m =22654653454549669696964832929654654646464653464646466356432
mprime = 53564365346534646785546786865765757868655346464758685746465354353525243242
for i in range(1,14):
    (c1,c2) = encRSA(m,20250514,N,N2,e)
    (c3,c4) = encRSA(mprime,20250514,N,N2,e)
    start = time.time()
    for j in range(2**i):
        c3,c4 = homomorphicAddRSA(c1,c2,c3,c4,N2)
    end = time.time()
    print(2**i, end - start)
    homomorphicAdditionTimes.append(end-start)

2 0.00011181831359863281
4 0.00021886825561523438
8 0.00043320655822753906
16 0.0008690357208251953
32 0.0017910003662109375
64 0.003493785858154297
128 0.0070421695709228516
256 0.013981819152832031
512 0.027745962142944336
1024 0.05549192428588867
2048 0.11087489128112793
4096 0.2215569019317627
8192 0.44260621070861816


In [78]:
print(homomorphicAdditionTimes)

[0.00011181831359863281, 0.00021886825561523438, 0.00043320655822753906, 0.0008690357208251953, 0.0017910003662109375, 0.003493785858154297, 0.0070421695709228516, 0.013981819152832031, 0.027745962142944336, 0.05549192428588867, 0.11087489128112793, 0.2215569019317627, 0.44260621070861816]


In [1]:
8192*1.94

15892.48

In [4]:
7.18/0.442

16.244343891402714

In [3]:
15892.48/2.01

7906.706467661692